<a href="https://colab.research.google.com/github/NikitasTsingenopoulos/Multi-XSience/blob/main/Multi_XScience.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U datasets
!pip install -U evaluate
!pip install rouge_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=9ddbdccdde3bafa1f0500b6f0f2e6c96677b208759e40294aea71fd36ca320fb
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [ ]:
from transformers import (AutoTokenizer, LEDConfig, LEDForConditionalGeneration)
from datasets import load_dataset,Dataset, DatasetDict
import torch

In [ ]:
!git clone https://github.com/yaolu/Multi-XScience.git

fatal: destination path 'Multi-XScience' already exists and is not an empty directory.


In [ ]:
!gunzip /content/Multi-XScience/data/test.json.gz
!gunzip /content/Multi-XScience/data/train.json.gz
!gunzip /content/Multi-XScience/data/val.json.gz

gzip: /content/Multi-XScience/data/test.json.gz: No such file or directory
gzip: /content/Multi-XScience/data/train.json.gz: No such file or directory
gzip: /content/Multi-XScience/data/val.json.gz: No such file or directory


In [ ]:
import json

train_path = '/content/Multi-XScience/data/train.json'
val_path = '/content/Multi-XScience/data/val.json'
test_path = '/content/Multi-XScience/data/test.json'


with open(train_path, 'r', encoding='utf-8') as f:
  dataset_train = json.load(f)

with open(val_path, 'r', encoding='utf-8') as f:
  dataset_val = json.load(f)

with open(test_path, 'r', encoding='utf-8') as f:
  dataset_test = json.load(f)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('allenai/PRIMERA')

config=LEDConfig.from_pretrained('allenai/PRIMERA')

model = LEDForConditionalGeneration.from_pretrained('allenai/PRIMERA')
model.gradient_checkpointing_enable()

PAD_TOKEN_ID = tokenizer.pad_token_id
DOCSEP_TOKEN_ID = tokenizer.convert_tokens_to_ids("<doc-sep>")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
def process_document(documents):
    input_ids_all = []
    for data in documents:
        article = data.replace("\n", " ")
        article = " ".join(article.split())
        input_ids = tokenizer.encode(
            article,
            truncation=True,
            max_length=4096,
        )[1:-1]
        input_ids = (
            [tokenizer.bos_token_id]
            + input_ids
            + [tokenizer.eos_token_id]
        )
        input_ids_all.append(torch.tensor(input_ids))

    input_ids = torch.nn.utils.rnn.pad_sequence(
        input_ids_all, batch_first=True, padding_value=PAD_TOKEN_ID
    )
    return input_ids

In [ ]:
def batch_process(batch):

    ref_abstracts = []
    for ref_dict in batch["ref_abstract"]:
        context_abstracts = []
        for inner in ref_dict.values():
            if isinstance(inner, dict) and "abstract" in inner:
                context_abstracts.append(inner["abstract"])
        # Join all abstracts for this example
        ref_abstracts.append(" ".join(context_abstracts))


    inputs = [a + " " + r
              for a, r in zip(batch['abstract'], ref_abstracts)
    ]
    input_ids=process_document(inputs)
    # get the input ids and attention masks together
    global_attention_mask = torch.zeros_like(input_ids).to(input_ids.device)
    # put global attention on <s> token

    global_attention_mask[:, 0] = 1
    global_attention_mask[input_ids == DOCSEP_TOKEN_ID] = 1
    generated_ids = model.generate(
        input_ids=input_ids,
        global_attention_mask=global_attention_mask,
        use_cache=True,
        max_length=1024,
        num_beams=5,
    )
    generated_str = tokenizer.batch_decode(
            generated_ids.tolist(), skip_special_tokens=True
        )

    result={}
    result['generated_summaries'] = generated_str
    result['gt_summaries']=batch['related_work'] # Use the correctly extracted abstracts for the batch
    return result

In [ ]:
import random

test_dataset = Dataset.from_list(dataset_test)
docs = random.choices(range(len(dataset_test)),k=5)
docs

[1906, 3562, 2356, 4050, 2975]

In [ ]:
dataset_small = test_dataset.select(docs)
result_small = dataset_small.map(batch_process, batched=True, batch_size=2)

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

Input ids are automatically padded from 943 to 1024 to be a multiple of `config.attention_window`: 512
Input ids are automatically padded from 880 to 1024 to be a multiple of `config.attention_window`: 512
Input ids are automatically padded from 843 to 1024 to be a multiple of `config.attention_window`: 512


In [ ]:
import evaluate

rouge = evaluate.load("rouge")

In [ ]:
import textwrap

for summ in result_small['generated_summaries']:
    wrapped_summ = textwrap.fill(summ, 160)
    print(wrapped_summ)
    print("\n")

Abstract In recent years, attention has been focused on the relationship between black-box optimization problem and reinforcement learning problem. A divergence
measure between two probability distributions or positive arrays (positive measures) is a useful tool for solving optimization problems in optimization, signal
processing, machine learning, and statistical inference. The Csiszar f-divergence is a unique class of divergences having information monotonicity, from which
the dual alpha geometrical structure with the Fisher metric is derived. The Kullback-Leibler divergence is proved to belong to both classes, and this is the
only such one in the space of probability distributions. This paper shows that the alpha-structure of the positive measures is the same as that of the negative
measures, and that the Bregman divergence is a special case of the alpha structure. We show that the F-diversions are the canonical divergence functions derived
from the dually flat geometry of the space

In [ ]:
score=rouge.compute(predictions=result_small["generated_summaries"], references=result_small["gt_summaries"])

print(f"ROUGE-1: {score['rouge1']:.4f}")
print(f"ROUGE-2: {score['rouge2']:.4f}")
print(f"ROUGE-L: {score['rougeL']:.4f}")

ROUGE-1: 0.2638
ROUGE-2: 0.0436
ROUGE-L: 0.1322


In [ ]:
example = dataset_small[4]

display(example)
#display(example_summ)

{'aid': '1701.01573',
 'mid': '2952318175',
 'abstract': "Smile is one of the key elements in identifying emotions and present state of mind of an individual. In this work, we propose a cluster of approaches to classify posed and spontaneous smiles using deep convolutional neural network (CNN) face features, local phase quantization (LPQ), dense optical flow and histogram of gradient (HOG). Eulerian Video Magnification (EVM) is used for micro-expression smile amplification along with three normalization procedures for distinguishing posed and spontaneous smiles. Although the deep CNN face model is trained with large number of face images, HOG features outperforms this model for overall face smile classification task. Using EVM to amplify micro-expressions did not have a significant impact on classification accuracy, while the normalizing facial features improved classification accuracy. Unlike many manual or semi-automatic methodologies, our approach aims to automatically classify all 